In [1]:
import sys
print(sys.path)

['/home/laurie2905/anaconda3/envs/masterEnv/lib/python310.zip', '/home/laurie2905/anaconda3/envs/masterEnv/lib/python3.10', '/home/laurie2905/anaconda3/envs/masterEnv/lib/python3.10/lib-dynload', '', '/home/laurie2905/.local/lib/python3.10/site-packages', '/home/laurie2905/anaconda3/envs/masterEnv/lib/python3.10/site-packages']


In [3]:
import process_data

ModuleNotFoundError: No module named 'process_data'

In [ ]:
import gymnasium as gym
from gymnasium import spaces
import numpy as np
import random

class CustomEnv(gym.Env):
    def __init__(self):
        # super so parent class is properly initialised
        super(CustomEnv, self).__init__()
        # Actions we can take: ingredient amount (out of 137 ingredients it chooses a quantity) 
        self.action_space = spaces.Discrete(3)
        # Temperature array
        self.observation_space = spaces.Box(low=np.array([0]), high=np.array([100]), dtype=np.float32)
        # Set start temp
        self.state = 38 + random.randint(-3, 3)
        # Set shower length
        self.shower_length = 60

    def step(self, action):
        # Apply action
        # 0 -1 = -1 temperature
        # 1 -1 = 0 
        # 2 -1 = 1 temperature 
        self.state += action - 1
        # Reduce shower length by 1 second
        self.shower_length -= 1

        # Calculate reward
        if 37 <= self.state <= 39:
            reward = 1
        else:
            reward = -1

        # Check if shower is done
        done = self.shower_length <= 0

        # Apply temperature noise
        self.state += random.randint(-1, 1)

        # Set placeholder for info
        info = {}

        # Return step information
        return np.array([self.state]).astype(float), reward, done, info

    def reset(self):
        # Reset shower temperature
        self.state = 38 + random.randint(-3, 3)
        # Reset shower time
        self.shower_length = 60
        return np.array([self.state]).astype(float)

    def render(self):
        # Implement visualization (optional)
        pass

    def close(self):
        # Clean up (optional)
        pass

# Example usage:
# env = ShowerEnv()
# observation = env.reset()
# done = False
# while not done:
#     action = env.action_space.sample()
#     observation, reward, done, info = env.step(action)
#     env.render()
# env.close()


In [3]:
from IPython.display import display, HTML

import pandas as pd
from numpy.random import randint

dict = {'Name':['Martha', 'Tim', 'Rob', 'Georgia'],
        'Maths':[87, 91, 97, 95],
        'Science':[83, 99, 84, 76]
       }

df = pd.DataFrame(dict)

display(df)

df.loc[len(df.index)] = ['Amy', 89, 93] 

display(df)

,Name,Maths,Science
0,Martha,87,83
1,Tim,91,99
2,Rob,97,84
3,Georgia,95,76


,Name,Maths,Science
0,Martha,87,83
1,Tim,91,99
2,Rob,97,84
3,Georgia,95,76
4,Amy,89,93


In [7]:
import os
# Define the base directory where your script is located
base_dir = os.path.dirname('customEnvTesting.ipynb')

# Construct the full path to the data.csv file
data_file_path = os.path.join(base_dir,'..','..', 'data', 'data.csv')

# Attempt to open and read the data.csv file
with open(data_file_path, 'r') as file:
    data = file.readlines()

# Process the data and report the number of rows and ingredients per row
num_rows = len(data)

print(f"Successfully read {num_rows} lines from the file. Loaded 137 ingredients.")

keys = data[0].replace('\n','').replace('\ufeff','').split(',')

ingredients_df = pd.DataFrame(columns=keys)

for i, row in enumerate(data):
    if i == 0:
        continue
    print(row)
    ingredient = row.replace('"','').replace('\n','').split(',')
    print(ingredient)
    print(len(ingredient))
    
    ingredients_df.loc[len(ingredients_df.index)] = ingredient

FileNotFoundError: [Errno 2] No such file or directory: '../../data/data.csv'